# Order DigiKey Components From BOM 
Python 3.10.16

This script generates a CSV BOM to upload to [DigiKey myLists](https://www.digikey.ca/en/mylists/): `BIODYN-100 vXX BOM.myLists.csv`.

In [60]:
%pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
# Read Tidy BOM
import pandas as pd
bom_file_name = "data/BIODYN-100 v58 BOM Tidy.csv"
outfile = "data/BIODYN-100 v58 BOM.myLists.csv"

df = pd.read_csv(bom_file_name, sep=";")
df.head(2)

,Part,Value,Device,Footprint Name,Detailed Description,Link,Manuf Part Number,Datasheet URL,Description,Manufacturer,DigiKey Part Number,Pricing,Stock,Package/Footprint
0,EMG1:D1,1N4148DO35-10,1N4148DO35-10,DO35-10,DIODE,https://www.digikey.ca/en/products/detail/onse...,1N4148,https://www.mccsemi.com/pdf/Products/1N4148WX(...,DIODE STANDARD 75V 150MA SOD323,MCC (Micro Commercial Components),1N4148WXTPMSCT-ND,"[{'break_quantity': 1, 'total_price': 0.1, 'un...",1138405.0,"SC-76, SOD-323"
1,EMG1:D2,1N4148DO35-10,1N4148DO35-10,DO35-10,DIODE,https://www.digikey.ca/en/products/detail/onse...,1N4148,https://www.mccsemi.com/pdf/Products/1N4148WX(...,DIODE STANDARD 75V 150MA SOD323,MCC (Micro Commercial Components),1N4148WXTPMSCT-ND,"[{'break_quantity': 1, 'total_price': 0.1, 'un...",1138405.0,"SC-76, SOD-323"


In [69]:

df['Count'] = 1
agg_funcs = {
	'Count': 'sum', 
	"Manufacturer": 'first',
	"Manuf Part Number": 'first',
	'DigiKey Part Number': 'first', 
	}
df_by_part = df.groupby(df['Manuf Part Number']).aggregate(agg_funcs)
df_by_part.rename(columns={'Count': 'Quantity', 
							 "Manuf Part Number": "Manufacturer Part Number",
							 "Manufacturer": "Manufacturer Name",
							 "DigiKey Part Number": "Requested Part Number"}, 
							 inplace=True)
df_by_part['Index'] = range(1, len(df_by_part) + 1)
df_by_part.set_index('Index', inplace=True)
new_order = ['Manufacturer Part Number', 'Manufacturer Name', 'Requested Part Number', 'Quantity'];
df_by_part = df_by_part[new_order]

df_by_part.head(5)

,Manufacturer Part Number,Manufacturer Name,Requested Part Number,Quantity
Index,,,,
1,1N4148,MCC (Micro Commercial Components),1N4148WXTPMSCT-ND,2
2,885012005024,Würth Elektronik,732-7440-1-ND,5
3,BQ21040DBVR,Texas Instruments,296-47315-1-ND,1
4,BQ29700DSER,Texas Instruments,296-43985-1-ND,1
5,C0402C150J8HACTU,KEMET,399-C0402C150J8HACTUCT-ND,2


In [63]:
df_by_part.to_csv(outfile, index=True)

# PCBWay BOM

In [76]:
agg_funcs = {
	'Count': 'sum', 
	"Manufacturer": 'first',
	"Manuf Part Number": 'first',
	'DigiKey Part Number': 'first',
	"Package/Footprint": 'first',
	'Description': 'first',
	'Part': lambda x: ', '.join(x.astype(str)),
	}
df_pcbway = df.groupby(df['Manuf Part Number']).aggregate(agg_funcs)
df_pcbway.rename(columns={"Manuf Part Number": "Mfg Part #",
						  'Part': "Designator",
						  "Count": "Qty",
						  "Link": "DigiKey Link"}, 
						  inplace=True)
new_order = ['Designator', 
			 'Qty', 
			 'Manufacturer', 
			 'Mfg Part #', 
			 'DigiKey Part Number', 
			 'Description',
			 "Package/Footprint"];
df_pcbway = df_pcbway[new_order]
df_pcbway['Item #'] = range(1, len(df_by_part) + 1)
df_pcbway['Type'] = "SMD"
df_pcbway['Notes'] = ""
df_pcbway.set_index('Item #', inplace=True)
df_pcbway = df_pcbway[df_pcbway['Mfg Part #'].notna()]

df_pcbway.head(10)

,Designator,Qty,Manufacturer,Mfg Part #,DigiKey Part Number,Description,Package/Footprint,Type,Notes
Item #,,,,,,,,,
1,"EMG1:D1, EMG1:D2",2,MCC (Micro Commercial Components),1N4148,1N4148WXTPMSCT-ND,DIODE STANDARD 75V 150MA SOD323,"SC-76, SOD-323",SMD,
2,"ESP1:C16, ESP1:C17, ESP1:C18, ESP1:C19, PWR1:9...",5,Würth Elektronik,885012005024,732-7440-1-ND,CAP CER 6.8PF 16V C0G/NP0 0402,0402 (1005 Metric),SMD,
3,PWR1:BATCHG1:U1,1,Texas Instruments,BQ21040DBVR,296-47315-1-ND,IC BATT CHG LI-ION 1CELL SOT23-6,SOT-23-6,SMD,
4,PWR1:BAT1:U2,1,Texas Instruments,BQ29700DSER,296-43985-1-ND,IC BATT PROT LI-ION 1CELL 6WSON,6-WFDFN,SMD,
5,"ESP1:C12, ESP1:C13",2,KEMET,C0402C150J8HACTU,399-C0402C150J8HACTUCT-ND,CAP CER 15PF 10V X8R 0402,0402 (1005 Metric),SMD,
6,PWR1:9VINVBSTBCK1:C13,1,KEMET,C0402C224K9PACTU,399-4886-1-ND,CAP CER 0.22UF 6.3V X5R 0402,0402 (1005 Metric),SMD,
7,"ESP1:C10, ESP1:C14",2,KEMET,C0603C106M9PACTU,399-C0603C106M9PACTUCT-ND,CAP CER 10UF 6.3V X5R 0603,0603 (1608 Metric),SMD,
8,PWR1:9VINVBSTBCK1:C10,1,KEMET,C0603C472F5GACTU,399-C0603C472F5GACTUCT-ND,CAP CER 4700PF 50V C0G/NP0 0603,0603 (1608 Metric),SMD,
9,EMG1:U$11,1,KEMET,C0805C103F5GECTU,399-C0805C103F5GECTUCT-ND,CAP CER 0805 10NF 50V C0G 1%,0805 (2012 Metric),SMD,


In [77]:
df_pcbway.to_csv("data/BIODYN-100 v64 BOM PCBWay.csv", index=False)


Copyright © 2025 Callum Mackenzie 